In [1]:
from models.waffleiron.segmenter import Segmenter
import torch
from datasets import LIST_DATASETS, Collate
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import argparse
import wandb
from torchmetrics.classification import MulticlassJaccardIndex

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

import torchhd
from torchhd.models import Centroid
from torchhd import embeddings

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))
device_string = "cuda:0" if torch.cuda.is_available() else "cpu"

model = Segmenter(
    input_channels=5,
    feat_channels=768,
    depth=48,
    grid_shape=[[256, 256], [256, 32], [256, 32]],
    nb_class=16, # class for prediction
    #drop_path_prob=config["waffleiron"]["drop_path"],
    layer_norm=True,
)

# Load pretrained model
ckpt = torch.load('/root/main/ScaLR/saved_models/ckpt_last_scalr.pth', map_location=device_string) # cuda:0
ckpt = ckpt["net"]

new_ckpt = {}
for k in ckpt.keys():
    if k.startswith("module"):
        if k.startswith("module.classif.0"):
            continue
        elif k.startswith("module.classif.1"):
            new_ckpt["classif" + k[len("module.classif.1") :]] = ckpt[k]
        else:
            new_ckpt[k[len("module.") :]] = ckpt[k]
    else:
        new_ckpt[k] = ckpt[k]

model.load_state_dict(new_ckpt)

if device_string != 'cpu':
    torch.cuda.set_device(device_string) # cuda:0
    model = model.cuda(device_string) # cuda:0

model.eval()

kwargs = {
        "rootdir": '/root/main/dataset/nuscenes',
        "input_feat": ["xyz", "intensity", "radius"],
        "voxel_size": 0.1,
        "num_neighbors": 16,
        "dim_proj": [2, 1, 0],
        "grids_shape": [[256, 256], [256, 32], [256, 32]],
        "fov_xyz": [[-64, -64, -8], [64, 64, 8]], # Check here
    }

# Get datatset
DATASET = LIST_DATASETS.get("nuscenes")

# Train dataset
train_dataset = DATASET(
    phase="train",
    **kwargs,
)

test_dataset = DATASET(
    phase="val",
    **kwargs,
)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(),
    )

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(),
    )

DIMENSIONS = 10000
FEAT_SIZE = 768
NUM_LEVELS = 1000
BATCH_SIZE = 1  # for GPUs with enough memory we can process multiple images at ones


class Encoder(nn.Module):
    def __init__(self, out_features, size, levels):
        super(Encoder, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.position = embeddings.Random(size, out_features)
        self.value = embeddings.Level(levels, out_features)

    def forward(self, x):
        sample_hv = torchhd.bind(self.position.weight, self.value(x))
        sample_hv = torchhd.multiset(sample_hv)
        return torchhd.hard_quantize(sample_hv)


encode = Encoder(DIMENSIONS, FEAT_SIZE, NUM_LEVELS)
encode = encode.to(device)

num_classes = 16
model_hd = Centroid(DIMENSIONS, num_classes)
model_hd = model_hd.to(device)

stop = 48

def forward_model(it, batch, stop):
    feat = batch["feat"]
    labels = batch["labels_orig"]
    cell_ind = batch["cell_ind"]
    occupied_cell = batch["occupied_cells"]
    neighbors_emb = batch["neighbors_emb"]
    if device_string != 'cpu':
        feat = feat.cuda(0, non_blocking=True)
        labels = labels.cuda(0, non_blocking=True)
        batch["upsample"] = [
            up.cuda(0, non_blocking=True) for up in batch["upsample"]
        ]
        cell_ind = cell_ind.cuda(0, non_blocking=True)
        occupied_cell = occupied_cell.cuda(0, non_blocking=True)
        neighbors_emb = neighbors_emb.cuda(0, non_blocking=True)

    with torch.no_grad():
        out = model(feat, cell_ind, occupied_cell, neighbors_emb, stop)
        embed, tokens = out[0][0], out[1][0]
        embed = embed.transpose(0, 1)
        tokens = tokens.transpose(0, 1)

        labels_v = [[] for i in range(embed.shape[0])]
        for i, vox in enumerate(batch["upsample"][0]):
            labels_v[vox].append(labels[i])
        labels_v_single = []
        for labels_ in labels_v:
            lab_tens = torch.tensor(labels_)
            most_common_value = torch.bincount(lab_tens).argmax()
            labels_v_single.append(most_common_value)
    
    return tokens, labels_v_single

for it, batch in enumerate(train_loader):
    
    # Network inputs
    
    tokens, labels_v_single = forward_model(it, batch, stop)

    print(tokens, labels_v_single)    

Using torch.scatter_reduce for 3D to 2D projection.


ModuleNotFoundError: No module named 'waffleiron'